<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE2-VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly.express as px
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import scipy.stats as stats
import pylab as pl

# Load generated data from VAE module
y = np.load(r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')
x = np.load(r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/generated-data2.npy')

print(f"Loaded data shapes - X: {x.shape}, Y: {y.shape}")

# Distribution visualization
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y, kde=True, color="red")
plt.show()

# Optional: Remove outliers (commented out as per original code)
# from sklearn.ensemble import IsolationForest
# iso = IsolationForest(contamination=0.4)
# yhat = iso.fit_predict(x)
# mask = yhat != -1
# x, y = x[mask, :], y[mask]

# Scale the target and split data
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()
y_transformed = transformer.fit_transform(y.reshape(-1, 1)).flatten()
x_train, x_test, y_train, y_test = train_test_split(x, y_transformed, test_size=0.1, random_state=42)

print(f"Train/Test split - X_train: {x_train.shape}, X_test: {x_test.shape}")
print(f"Train/Test split - y_train: {y_train.shape}, y_test: {y_test.shape}")

# Set up model checkpointing
checkpoint_path = "/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/ckp2.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='loss',
    save_best_only=True,
    save_weights_only=True
)

# Build MLP model
from keras.layers import LeakyReLU

neuron = 64
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

optimizr = keras.optimizers.Adam(learning_rate=0.0003, clipnorm=1)
model.compile(loss='mean_squared_error', optimizer=optimizr, metrics=['mean_squared_error'])

# Initial build
model.fit(x_train, y_train,
          epochs=1,
          batch_size=32,
          callbacks=[cp_callback])

model.load_weights(checkpoint_path)

# Set up callbacks
es = keras.callbacks.EarlyStopping(
    patience=20,
    verbose=1,
    min_delta=0.0001,
    monitor='loss',
    mode='min',
    restore_best_weights=True
)
n_epochs = 1000

# Train the model
history = model.fit(x_train, y_train,
                   epochs=n_epochs,
                   batch_size=32,
                   validation_split=0.1,
                   callbacks=[cp_callback])

# Plot training history
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Training evaluation
y_train_pred_raw = model.predict(x_train)
y_train_pred = transformer.inverse_transform(y_train_pred_raw)
y_train_true = transformer.inverse_transform(y_train.reshape(-1, 1)).flatten()

score_train = r2_score(y_train_true, y_train_pred)
print("Training r2 score is ==", score_train)

plt.plot(y_train_true[0:100], color='red', label='Real data')
plt.plot(y_train_pred[0:100], color='blue', label='Predicted data')
plt.title('Training Prediction')
plt.legend()
plt.show()

# Test evaluation
y_pred_raw = model.predict(x_test)
y_test_pred = transformer.inverse_transform(y_pred_raw)
y_test_true = transformer.inverse_transform(y_test.reshape(-1, 1)).flatten()

score = r2_score(y_test_true, y_test_pred)
print("Test r2 score is ==", score)

plt.plot(y_test_true[100:200], color='red', label='Real data')
plt.plot(y_test_pred[100:200], color='blue', label='Predicted data')
plt.title('Test Prediction')
plt.legend()
plt.show()

# Save model and results
model.save(r'/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras')

# Final training history plot
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Load and test saved model
base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2025/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras')

# Final test evaluation with loaded model
y_pred_raw = base_model.predict(x_test)
y_test_pred = transformer.inverse_transform(y_pred_raw)
y_test_true = transformer.inverse_transform(y_test.reshape(-1, 1)).flatten()

score = r2_score(y_test_true, y_test_pred)
print("Final test r2 score with loaded model ==", score)

plt.plot(y_test_true[100:200], color='red', label='Real data')
plt.plot(y_test_pred[100:200], color='blue', label='Predicted data')
plt.title('Final Prediction with Loaded Model')
plt.legend()
plt.show()

# Save results
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/METROPM_predicted_window.csv', y_test_pred)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/METROPM_derived_window_label.csv', y_test_true)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/METROPM_test_data.csv', x_test)

print("MLP training complete. Model and results saved.")

114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')